### Importing Essential Packages

In [1]:
import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Loading and preparing the dataset 

In [2]:
X, y = datasets.load_iris(return_X_y=True)

In [3]:
X.shape, y.shape

((150, 4), (150,))

In [4]:
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [5]:
y[:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [6]:
X = StandardScaler().fit_transform(X)
# y = StandardScaler().fit_transform(np.expand_dims(y, -1))

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.2,
                                                    random_state=42)


X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train).long()
X_test, y_test = torch.tensor(X_test).float(), torch.tensor(y_test).long()

In [7]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([120, 4]),
 torch.Size([120]),
 torch.Size([30, 4]),
 torch.Size([30]))

In [8]:
ds_train = torch.utils.data.TensorDataset(X_train, y_train)
dataloader_train = torch.utils.data.DataLoader(ds_train, batch_size=16, shuffle=True)

ds_test = torch.utils.data.TensorDataset(X_test, y_test)
dataloader_test = torch.utils.data.DataLoader(ds_test, batch_size=16, shuffle=True)

### Defining a simple Bayesian model

prior_mu (Float) is the mean of prior normal distribution.

prior_sigma (Float) is the sigma of prior normal distribution.

In [9]:
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=4, out_features=100),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=100, out_features=3),
)

### Defining loss function and optimizer

The two-loss functions used here are cross-entropy loss and the BKL loss which is used to compute the KL (Kullback–Leibler) divergence of the network.

In [10]:
cross_entropy_loss = nn.CrossEntropyLoss()
klloss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
klweight = 0.01
optimizer = optim.Adam(model.parameters(), lr=0.01)

### Defining a confidence interval evaluating function

In [73]:
def evaluate_model(model,
                        X,
                        y,
                        samples = 100):
    model.eval()
    preds = [model(X) for i in range(samples)]
    preds = torch.stack(preds)
    
    means = preds.mean(axis=0)
    stds = preds.std(axis=0)
    
    _, predicted = torch.max(means, 1)
    correct = (predicted == y_test).sum()
    accuracy = 100 * float(correct) / predicted.size(0)
    
    return accuracy, means, stds

### Training the model

The model is trained for 3000 steps(this would have lead to overfitting for a traditional network)

In [76]:
iteration = 0
for epoch in range(100):
    for i, (datapoints, labels) in enumerate(dataloader_train):
        optimizer.zero_grad()
        model.train()
        preds = model(datapoints)
        cross_entropy = cross_entropy_loss(preds, labels)
        kl = klloss(model)
        total_loss = cross_entropy + klweight*kl

        total_loss.backward()
        optimizer.step()
        
        iteration += 1
        if iteration%100==0:
            accuracy, means, stds = evaluate_model(model,
                                                    X_test,
                                                    y_test,
                                                    samples=25)
            
            print("Eval Accuracy: {:.2f}".format(accuracy))
            print("Training Loss: {:.4f}".format(total_loss.detach().numpy()[0]))
            print()

Eval Accuracy: 100.00
Training Loss: 0.0584

Eval Accuracy: 100.00
Training Loss: 0.0896

Eval Accuracy: 100.00
Training Loss: 0.0374

Eval Accuracy: 100.00
Training Loss: 0.0376

Eval Accuracy: 100.00
Training Loss: 0.1585

Eval Accuracy: 100.00
Training Loss: 0.1448

Eval Accuracy: 100.00
Training Loss: 0.0374

Eval Accuracy: 96.67
Training Loss: 0.0334



Source: https://analyticsindiamag.com/hands-on-guide-to-bayesian-neural-network-in-classification/